In [100]:
import cv2
import numpy as np

In [101]:
#deep nueral network'
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

In [124]:
classes=[]
with open('coco.names','r') as f:
    classes=[line.strip() for line in f.readlines()]# convert line into strip
print(classes)
colors = np.random.uniform(0,255,size=(len(classes),3))

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [103]:
len(classes)# 80 classes are present

80

In [104]:
layer_name=net.getLayerNames()#layers
print(layer_name)

['conv_0', 'bn_0', 'leaky_1', 'conv_1', 'bn_1', 'leaky_2', 'conv_2', 'bn_2', 'leaky_3', 'conv_3', 'bn_3', 'leaky_4', 'shortcut_4', 'conv_5', 'bn_5', 'leaky_6', 'conv_6', 'bn_6', 'leaky_7', 'conv_7', 'bn_7', 'leaky_8', 'shortcut_8', 'conv_9', 'bn_9', 'leaky_10', 'conv_10', 'bn_10', 'leaky_11', 'shortcut_11', 'conv_12', 'bn_12', 'leaky_13', 'conv_13', 'bn_13', 'leaky_14', 'conv_14', 'bn_14', 'leaky_15', 'shortcut_15', 'conv_16', 'bn_16', 'leaky_17', 'conv_17', 'bn_17', 'leaky_18', 'shortcut_18', 'conv_19', 'bn_19', 'leaky_20', 'conv_20', 'bn_20', 'leaky_21', 'shortcut_21', 'conv_22', 'bn_22', 'leaky_23', 'conv_23', 'bn_23', 'leaky_24', 'shortcut_24', 'conv_25', 'bn_25', 'leaky_26', 'conv_26', 'bn_26', 'leaky_27', 'shortcut_27', 'conv_28', 'bn_28', 'leaky_29', 'conv_29', 'bn_29', 'leaky_30', 'shortcut_30', 'conv_31', 'bn_31', 'leaky_32', 'conv_32', 'bn_32', 'leaky_33', 'shortcut_33', 'conv_34', 'bn_34', 'leaky_35', 'conv_35', 'bn_35', 'leaky_36', 'shortcut_36', 'conv_37', 'bn_37', 'leaky_

In [105]:
for i in net.getUnconnectedOutLayers():#output layers
    print(i)

[200]
[227]
[254]


In [106]:
output_layer=[layer_name[i[0]-1] for i in net.getUnconnectedOutLayers()]
print(output_layer)

['yolo_82', 'yolo_94', 'yolo_106']


In [107]:
image=cv2.imread("cycle3.jpeg")
print(image.shape)
image=cv2.resize(image,None,fx=0.4,fy=0.4)
height,width,channel=image.shape

(750, 500, 3)


In [108]:
image.shape#h,w,c


(300, 200, 3)

In [109]:
blob = cv2.dnn.blobFromImage(image,scalefactor=0.00392,size=(416,416),mean=(0,0,0),swapRB=True,crop=False)
#blob = cv2.dnn.blobFromImages(images, scalefactor=1.0, size, mean, swapRB=True)

In [110]:
blob[0][0].shape


(416, 416)

In [111]:
cv2.imshow('image',blob[0][2])
cv2.waitKey(0)
cv2.destroyAllWindows()


In [112]:
net.setInput(blob)#trained model
outs=net.forward(output_layer)

In [115]:
class_ids = []
confidences = []
boxes=[]
for out in outs:
    for detection in out:
        score=detection[5:]
        class_id=np.argmax(score)
        confidence = score[class_id]
        #print(confidence)
        if confidence > 0.5:
            center_x =int(detection[0]*width)
            center_y=int(detection[1]*height)
            w=int(detection[2]*width)
            h=int(detection[3]*height)
            ## Rectangle coordinates
            x = int(center_x - w/2)
            y = int(center_y - h/2)
            boxes.append([x,y,w,h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

In [116]:
np.array(confidences).dtype

dtype('float64')

In [117]:
indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.4)
#cv.Net.NMSBoxes(bboxes, scores, score_threshold, nms_threshold)

In [118]:
indexes

array([[2],
       [6]], dtype=int32)

In [119]:
len(indexes)

2

In [120]:
image2 = image.copy()

In [121]:
cv2.imshow('image',image2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [125]:
for i in range(len(boxes)):
    if i in indexes:
        x,y,w,h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[class_ids[i]]
        print(x,y,w,h)
        cv2.rectangle(image2,(x,y),(x+w,y+h),color,3)
        cv2.putText(image2,label,(x,y-20),cv2.FONT_HERSHEY_COMPLEX,1,color,1)


69 86 68 160
36 167 133 98


In [126]:
cv2.imshow('image',image2)
cv2.waitKey(0)
cv2.destroyAllWindows()